## words as the demonstration
api call

In [ ]:
import random
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff
from parsing import postprocessing

api_key = "sk-mE91TMZY8yikxpif8fBa64F0BaBa4d76BcCdD0Cb13F437D2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content.strip('\n')
    # return response

logs = ['proxy.cse.cuhk.edu.hk:5070 open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS',
        '183.62.156.108:22 open through proxy socks.cse.cuhk.edu.hk:5070 SOCKS5',
        '223.167.104.147:80 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - Could not resolve proxy.cse.cuhk.edu.hk error 11001',
        'qa.sockets.stackexchange.com:443 error : Could not connect to proxy proxy.cse.cuhk.edu.hk:5070 - connection attempt failed with error 10061',
        'mtalk.google.com:5228 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection',
        'tcpconn4.tencent.com:80 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy closed the connection unexpectedly.',
        'tcpconn6.tencent.com:443 error : A connection request was canceled before the completion.',
        'proxy.cse.cuhk.edu.hk:5070 close, 2933 bytes (2.86 KB) sent, 11721005 bytes (11.1 MB) received, lifetime 02:48',
        'proxy.cse.cuhk.edu.hk:5070 close, 451 bytes sent, 353 bytes received, lifetime <1 sec']

prompt1 = '''You will be provided with some varaibles, you must discard the variables that are similar or of the same type.
print the remaining variables in a list delimited by backticks.
['183.62.156.108:22', 'socks.cse.cuhk.edu.hk:5070', 'proxy.cse.cuhk.edu.hk:5070', '403 bytes', '426 bytes', '<1 sec', 'video-hkg3-2.xx.fbcdn.net:443', '58373 bytes (57.0 KB)', '8896991 bytes (8.48 MB)', '02:25', 'tcpconn6.tencent.com:443', '182.254.114.110:80']'''

prompt = '''You will be provided with some varaibles and a log message delimited by backticks. You must abstract variables with `{{placeholders}}` to extract the corresponding template.
Print the input log's template delimited by backticks.
varaibles:
`['183.62.156.108:22', 'socks.cse.cuhk.edu.hk:5070', 'proxy.cse.cuhk.edu.hk:5070', '403 bytes', '426 bytes', '<1 sec', 'video-hkg3-2.xx.fbcdn.net:443', '58373 bytes (57.0 KB)', '8896991 bytes (8.48 MB)', '02:25', 'tcpconn6.tencent.com:443', '182.254.114.110:80']`
log message:
'''
for log in logs:
    messages = [
        {'role': 'user', 'content': f"{prompt}`{log}`"},
    ]
    # 
    print(postprocessing(get_responce(messages)))

# print(get_responce([{'role': 'user', 'content': prompt1}]))

## DPP to sample the words from logs

In [ ]:
import pandas as pd
import re
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_csv('dataset\Proxifier\Proxifier_2k.log_structured_corrected.csv')
def dpp_sample(S, k):
    # S: similarity matrix
    # k: number of items to sample
    n = S.shape[0]
    
    # Initialize empty set Y
    Y = set()
    
    for _ in range(k):
        best_i = -1
        best_p = -1
        
        for i in range(n):
            if i not in Y:
                # Compute determinant of submatrix
                det_Yi = np.linalg.det(S[np.ix_(list(Y) + [i], list(Y) + [i])])
                
                # Compute probability of adding i to Y
                p_add = det_Yi / (1 + det_Yi)
                
                if p_add > best_p:
                    best_p = p_add
                    best_i = i
        
        # Add best item to Y
        Y.add(best_i)
    
    return list(Y)
def extract_variables(log, template):
    # 将模板中的 <*> 替换为正则表达式的捕获组 (.*?)
    # 为了避免正则表达式的特殊字符导致的问题，先将模板中除了 <*> 外的其他部分进行转义
    # 然后将 <*> 替换为正则表达式的捕获组
    # 这里假设模板中的 <*> 不紧邻正则特殊字符，如果有，需要更复杂的处理
    pattern_parts = template.split("<*>")
    pattern_parts_escaped = [re.escape(part) for part in pattern_parts]
    regex_pattern = "(.*?)".join(pattern_parts_escaped)
    regex = "^" + regex_pattern + "$"  # 添加开始和结束锚点以确保完整匹配

    matches = re.search(regex, log)
    if matches:
        return matches.groups()
    else:
        return []

logs = df['Content'].tolist()
templates = df['EventTemplate'].tolist()
# vars = []
# for log, template in zip(logs, templates):
#     variables = extract_variables(log, template)
#     for var in variables:
#         if var not in vars and "proxy.cse.cuhk.edu.hk" not in var and "bytes" not in var:
#             vars.append(var)
# print(len(vars))
# for var in vars:
#     print(var)  

# 使用 TF-IDF 向量化
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(logs)  # logs 是你的文本日志列表

# 转换为 numpy array 以便后续计算
tfidf_matrix = tfidf_matrix.toarray()

vars = []
# 计算余弦相似度矩阵
similarity_matrix = cosine_similarity(tfidf_matrix)
# result = dpp_sample(similarity_matrix, 5)
# 设置固定的随机种子
random.seed(0)

# 从1到2000中选择5个不重复的随机数
result = random.sample(range(0, 2000), 5)

print(result)
for i in result:
    variables = extract_variables(logs[i], templates[i])
    for var in variables:
        if var not in vars:
            vars.append(var)
    print(logs[i])
print(vars)
# proxy.cse.cuhk.edu.hk:5070
# 
# 403 bytes

## cross project test

In [ ]:
# discard the target dataset
dataset = 'Apache'
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
            'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
datasets.remove(dataset)
global demonstration_templates
global demonstration_logs
demonstration_templates = []
demonstration_logs = []
for d in datasets:
    df = pd.read_csv(f'dataset\{d}\{d}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template  in zip(list1, list2):
        if template not in demonstration_templates:
            demonstration_templates.append(template)
            demonstration_logs.append(log)


In [ ]:
import difflib


def find_closest_matches(target, list1, list2, n=5):
    matches = difflib.get_close_matches(target, list1, n=n,cutoff=0.1)
    indices = [list1.index(match) for match in matches]
    return [list1[index] for index in indices],[list2[index] for index in indices]

list1, list2 = find_closest_matches('PacketResponder 1 for block blk_38865049064139660 terminating', demonstration_templates, demonstration_logs)

print(list1)
print(list2)

验证按' '分词有多少词，以及含数字的词占多少

In [ ]:
import re
import pandas as pd
from post_process import correct_single_template
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Android','Hadoop', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# datasets = ['Linux']


def tokenize(log_content, tokenize_pattern=r'[ ,]'):
    words = re.split(tokenize_pattern, log_content)
    for index, word in enumerate(words):
        if word.startswith('/') and len(word) > 1:
            words[index] = ''
        if '=' in word:
            words[index] = word.split('=')[0]
        if re.search(r'\d', word):
            words[index] = ''
    words = [word for word in words if word]   # remove null
    return words


a = 0
b = 0
list = []
list2 = []
k = 0
temp = ['structured', 'templates', 'Content', 'EventTemplate']
tokens = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv")
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    print('-' * 20)
    print(dataset)

    # 验证长度为1-3的tempalte是否含有<*>
    # NOTE: 长度为1时，不用解析，2-3仍然需要解析
    # DS
    # NOTE: 没有template中含有'  '
    # BL+US
    # NOTE: boolen变量主要出现在Android中 null:Mac, Android root:many datasets admin:OpenSSH, Thunderbird
    # BL = ['true', 'false']
    # US = ['null', 'root', 'admin']
    # DG
    # NOTE: 没有template含有纯数字
    #
    tem = []
    for log, template in zip(logs, templates):
        if '/' in log and template not in tem:
            words = tokenize(log)
            tem.append(template)
            print(words)
            print(template)
            print('-' * 20)
            
            # print('-' * 20)
    # 验证按' '分词有多少词，以及含数字的词占多少
    # list2.append(len(list))    
    # total_words = 0
    # words_with_numbers = 0

    # for s in list:
    #     words = s.split()
    #     total_words += len(words)
    #     words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    # a+=total_words
    # b+=words_with_numbers


# print(f"{b} / {a}")

In [ ]:
from Levenshtein import distance


def closest_template(target, templates):
    min_distance = float('inf')
    closest_template = None
    for template in templates:
        d = distance(target, template)
        if d < min_distance:
            min_distance = d
            closest_template = template
    return closest_template

templates = [
    '<*> close, <*> bytes sent, <*> bytes received, lifetime <1',
    'proxy.cse.cuhk.edu.hk:<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1',
    '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>',
    '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> MB) received, lifetime <*>',
]

print(closest_template('proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01', templates))

生成候选的labelled logs

In [ ]:
import json
import pandas as pd
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')

data = {}
templates = []
logs = []
for dataset in datasets:
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template  in zip(list1, list2):
        if template not in templates:
            templates.append(template)
            logs.append(log)
    # 'logs': logs_c, 'templates': templates_c,
    print(dataset, len(logs))


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(target, lst, n=3):
    vectors = [np.array([ord(c) for c in s]).reshape(1, -1) for s in lst]
    target_vector = np.array([ord(c) for c in target]).reshape(1, -1)
    similarities = cosine_similarity(vectors, target_vector)
    most_similar_indices = np.argsort(similarities, axis=0)[-n:].flatten()[::-1]
    return [lst[i] for i in most_similar_indices]

In [ ]:
import pandas as pd
import re
# read log
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print(dataset)
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    for log,template in zip(logs, templates):    
        if 'kb' in log.lower() and template not in list:
            print(log)
            list.append(template)


list = ['sec', 'KB']

查看logpub中oracle template不合理的部分

In [ ]:
from post_process import correct_single_template
import pandas as pd
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print('+' * 20 + dataset)
    df = pd.read_csv(
        f'dataset\{dataset}\{dataset}_2k.log_templates_corrected.csv')
    templates = df['EventTemplate'].tolist()
    for template in templates:
        if correct_single_template(template) != template:
            print(correct_single_template(template))
            print(template)
            print('-' * 20)

try to conclude the types of variables

In [ ]:
import pandas as pd 
import re

def extract_variables(log, template):
    # 将模板中的 <*> 替换为正则表达式的捕获组 (.*?)
    # 为了避免正则表达式的特殊字符导致的问题，先将模板中除了 <*> 外的其他部分进行转义
    # 然后将 <*> 替换为正则表达式的捕获组
    # 这里假设模板中的 <*> 不紧邻正则特殊字符，如果有，需要更复杂的处理
    pattern_parts = template.split("<*>")
    pattern_parts_escaped = [re.escape(part) for part in pattern_parts]
    regex_pattern = "(.*?)".join(pattern_parts_escaped)
    regex = "^" + regex_pattern + "$"  # 添加开始和结束锚点以确保完整匹配

    matches = re.search(regex, log)
    if matches:
        return matches.groups()
    else:
        return []

def check_variable(variable):
    variable = variable.strip()
    if variable.startswith('/'):
        return 'path'
    if variable.startswith('0x'):
        return 'address'
    if re.match(r'\b-?\d+(\.\d+)?\b|\b0x[0-9a-fA-F]+\b', variable):
        return 'number'
    if re.match( r'^[a-zA-Z]+$', variable):
        return 'word'
    if re.match(  r'\b(?:\d{1,3}\.){3}\d{1,3}(?::\d{1,5})?\b', variable):
        return 'ip'
    else:
        return 'null'

table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')

dict = {}

for dataset in datasets:
    dict[dataset] = {}
    print('-' * 20)
    print(dataset + ':')
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    been_used = []
    for log, template in zip(logs, templates):
        if template not in been_used:
            been_used.append(template)
            variables = extract_variables(log, template)
            for variable in variables:
                type = check_variable(variable)
                if type != 'null':
                    if type not in dict[dataset]:
                        dict[dataset][type] = 1
                    else:
                        dict[dataset][type] += 1
                else:
                    print(variable)

print(dict)

Test the unseen accurray

In [ ]:
from collections import Counter
from evaluator import evaluate
import os
import pandas as pd
from sklearn.metrics import accuracy_score

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
# table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Linux HealthApp Apache Proxifier OpenSSH OpenStack Mac' # logpub
datasets = table_order.split(' ')
a = 0
b = 0
for dataset in datasets:
    file = '0125_0shot'
    input = f'outputs/parser/{file}/{dataset}.csv'  
    output = f'outputs/unseen/{file}/{dataset}.csv'
    os.makedirs(f'outputs/unseen/{file}', exist_ok=True)
    df = pd.read_csv(input)
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    freq = Counter(templates)
    logs_after = []
    templates_after = []
    unseen_templates = [item for item, count in freq.items() if count == 1]
    for log, template in zip(logs, templates):
        if template in unseen_templates:
            logs_after.append(log)
            templates_after.append(template)
    accuracy_exact_string_matching = accuracy_score(templates_after, logs_after, normalize=False)
    length = len(logs_after)
    a += length
    b += accuracy_exact_string_matching
    dataset = ' ' * (12 - len(dataset)) + dataset
    print('%s: len of unseen log: %.4d, Message-Level Accuracy: %.4f' %(dataset, length ,accuracy_exact_string_matching/length))
    

print(b/a)

# 81.0 71.2